In [1]:
import cv2
import numpy as np
import typing
from matplotlib import pyplot as plt

import os

In [2]:
working_p = "./dataset/working/"
in_p = working_p + "in/"
out_p = working_p + "out/"

# import from in_p folder all file names as paths
files = [f for f in os.listdir(in_p)]
# print(files)

In [4]:
def scale_to(img, size: int, interpolation=cv2.INTER_AREA):
    """
    Resize square image to given size (in pixels)
    """
    return cv2.resize(img, (size, size), interpolation=cv2.INTER_AREA)

In [125]:
def get_tire_circles(img, min_r: int, max_r: int, min_dist: int):
    """
    Find the outter circle and the inner circle in an image with a tire
    1. find the first circle in the image
    2. find all sequend circles in the image adjusting min_r with 10 higher iver the radius of the found circle
    3. remove circles that are very simmilar to each other
    4. select the group of circles with centers close to each other
    5. select the circle with the highest radius

    Note: min_dist should be img size so we find just a circle in the image
    """

    # 1. find the first circle in the image
    circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 2,
                                param1=170, param2=100,
                                minDist=min_dist,
                                minRadius=min_r, maxRadius=max_r)

    if circles is None:
        circles = np.array([[[img.shape[0] / 2, img.shape[1] / 2, min(img.shape[0], img.shape[1]) / 3]]])
        print("No circles found, substituting with center of image")

    start_circle = circles[0][0]
    start_circle = np.uint16(np.around(start_circle))

    circle_l: typing.Set[typing.Tuple[int]] = set()
    circle_l.add(tuple(start_circle))
    
    # print("circle_l: ", circle_l, end="\n\n")

    # 2. find all sequend circles in the image adjusting min_r with 10 higher iver the radius of the found circle
    for new_min_r in range(start_circle[2] + 10, max_r - 10, 10):
        circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 2,
                                    param1=170, param2=100,
                                    minDist=min_dist,
                                    minRadius=new_min_r, maxRadius=max_r)
        if circles is not None:
            circle = circles[0][0]
            circle = np.uint16(np.around(circle))
            circle_l.add(tuple(circle))

    for new_max_r in range(start_circle[2] - 10, min_r + 10, -10):
        circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 2,
                                    param1=170, param2=100,
                                    minDist=min_dist,
                                    minRadius=min_r, maxRadius=new_max_r)
        if circles is not None:
            circle = circles[0][0]
            circle = np.uint16(np.around(circle))
            circle_l.add(tuple(circle))

    # 3. remove circles that are very simmilar to each other

    # print("circle_l: ", circle_l)
    return list(circle_l)
    

In [127]:
for file_name in files:

    # read image
    img_path = in_p + file_name
    img_BGR = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)

    BGR_small = scale_to(img_BGR, 300)

    gray_small = cv2.cvtColor(BGR_small, cv2.COLOR_BGR2GRAY)

    # smooth the image (aka remove noise by blur)
    gray_small_blur = cv2.GaussianBlur(gray_small, (5, 5), cv2.BORDER_DEFAULT)
    # Canny edge detection
    gray_small_blur_canny = cv2.Canny(gray_small_blur, 75, 150, L2gradient=True)

    cv2.imwrite(out_p + file_name + "-0", gray_small_blur_canny)

    # -------------------------------
    # Circle detection
    # -------------------------------
    # outer radius = 140
    # inner radius = 75
    # circles = cv2.HoughCircles(gray_small_blur_canny,
    #                             cv2.HOUGH_GRADIENT, 2, minDist=300,
    #                             param1=50, param2=30,
    #                             minRadius=75, maxRadius=140)

    # output_img = BGR_small.copy()

    # if circles is not None:
    #     # convert the (x, y) coordinates and radius of the circles to integers
    #     circles = np.round(circles[0, :]).astype("int")

    #     # circles = sorted(circles, key=lambda x: x[2])
    #     print(file_name, "->", circles)

    #     # loop over the (x, y) coordinates and radius of the circles
    #     for (x, y, r) in circles:
    #         # draw the circle in the output image, then draw a rectangle
    #         # corresponding to the center of the circle
    #         cv2.circle(output_img, (x, y), r, (0, 255, 0), 4)
    #         cv2.rectangle(output_img, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)

    # cv2.imwrite(out_p + file_name + "-1", output_img)


    # -------------------------------
    output_img = BGR_small.copy()
    circles = get_tire_circles(gray_small_blur, 75, 140, 300)
    print("circles of", file_name, ":", circles)
    for circle in circles:
        # print(file_name, "->", circle)
        cv2.circle(output_img, (circle[0], circle[1]), circle[2], (0, 255, 0), 2)
        cv2.rectangle(output_img, (circle[0] - 5, circle[1] - 5), (circle[0] + 5, circle[1] + 5), (0, 128, 255), 1)

    cv2.imwrite(out_p + file_name + "-1", output_img)

    # break # only for testing

circles of 20211021_130952.jpg : [(147, 151, 91), (157, 155, 137)]
circles of 20211021_131012.jpg : [(153, 141, 98)]
circles of 20211021_131031.jpg : [(155, 155, 76), (157, 149, 108)]
circles of 20211021_131047.jpg : [(155, 153, 112), (157, 151, 112), (153, 153, 81)]
circles of 20211021_131709.jpg : [(145, 151, 80), (147, 147, 116)]
circles of 20211021_131724.jpg : [(153, 143, 109), (151, 153, 77), (153, 147, 112), (153, 141, 108), (155, 123, 139)]
circles of 20211021_131736.jpg : [(151, 151, 86), (155, 143, 119)]
circles of 20211021_131747.jpg : [(151, 149, 79), (149, 137, 117)]
circles of 20211021_132824.jpg : [(151, 145, 128), (153, 149, 87), (155, 139, 121)]
circles of 20211021_132904.jpg : [(143, 151, 92), (147, 145, 132), (151, 139, 126), (147, 147, 134)]
circles of 20211021_132922.jpg : [(155, 127, 76), (159, 123, 110), (159, 121, 108), (153, 95, 134)]
circles of 20211021_132941.jpg : [(155, 153, 89), (155, 161, 125), (153, 155, 89)]
circles of 20211021_133217.jpg : [(161, 145, 